### Parametros Iniciais 

In [0]:
%pip install openpyxl
import os, io, csv, hashlib, datetime, re, requests
import pandas as pd
from io import BytesIO
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from pyspark.sql import functions as F, types as T

CATALOG        = "sinesp"
SCHEMA_BRONZE  = "bronze"
LANDING_UF  = "/Volumes/sinesp/source/landing/sinesp/uf"

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA_BRONZE}")

### Localizar XLSX mais recente na landing

In [0]:
df_files = (spark.read.format("binaryFile")
            .option("recursiveFileLookup","true")
            .load(LANDING_UF)
            .filter(F.lower(F.col("path")).endswith(".xlsx"))
            .select("path","modificationTime"))

latest = df_files.orderBy(F.col("modificationTime").desc()).limit(1).collect()
if not latest:
    raise FileNotFoundError(f"Nenhum XLSX encontrado em {LANDING_UF}")

latest_path = latest[0]["path"]
print("Usando arquivo UF:", latest_path)

### Abrir XLSX via Spark -> Bytes -> pandas

In [0]:
file_bytes = (spark.read.format("binaryFile")
              .load(latest_path)
              .select("content")
              .head()[0])

xls = pd.ExcelFile(BytesIO(file_bytes))
print("Abas encontradas:", xls.sheet_names)

### Helpers de normalização

In [0]:
def canon(s: str) -> str:
    x = s.strip().lower()
    x = (x.replace("ã","a").replace("á","a").replace("â","a").replace("à","a")
           .replace("é","e").replace("ê","e")
           .replace("í","i")
           .replace("ó","o").replace("ô","o")
           .replace("ú","u").replace("ü","u")
           .replace("ç","c"))
    return re.sub(r"[^a-z0-9_]+","_", x)

def find_sheet(xls: pd.ExcelFile, wanted: str):
    w = canon(wanted)
    for s in xls.sheet_names:
        if canon(s) == w:
            return s
    # tentativas mais soltas (ocorrencias/vitimas sem acento)
    for s in xls.sheet_names:
        cs = canon(s)
        if w in cs:
            return s
    return None

MONTH_MAP = {
    "janeiro":1,"fevereiro":2,"marco":3,"março":3,"abril":4,"maio":5,"junho":6,
    "julho":7,"agosto":8,"setembro":9,"outubro":10,"novembro":11,"dezembro":12,
    "jan":1,"fev":2,"mar":3,"abr":4,"mai":5,"jun":6,"jul":7,"ago":8,"set":9,"out":10,"nov":11,"dez":12,
    "jan.":1,"fev.":2,"mar.":3,"abr.":4,"mai.":5,"jun.":6,"jul.":7,"ago.":8,"set.":9,"out.":10,"nov.":11,"dez.":12
}

def month_from_text(x):
    if pd.isna(x): return pd.NA
    s = str(x).strip().lower()
    # se vier como data
    try:
        ts = pd.to_datetime(x, errors="coerce")
        if pd.notna(ts):
            return int(ts.month)
    except Exception:
        pass
    # busca no dicionário
    return MONTH_MAP.get(s, pd.NA)

UF_SIGLA = {
    "acre":"AC","alagoas":"AL","amapa":"AP","amapá":"AP","amazonas":"AM","bahia":"BA","ceara":"CE","ceará":"CE",
    "distrito federal":"DF","espirito santo":"ES","espírito santo":"ES","goias":"GO","goiás":"GO","maranhao":"MA","maranhão":"MA",
    "mato grosso":"MT","mato grosso do sul":"MS","minas gerais":"MG","para":"PA","pará":"PA","paraiba":"PB","paraíba":"PB",
    "parana":"PR","paraná":"PR","pernambuco":"PE","piaui":"PI","piauí":"PI","rio de janeiro":"RJ","rio grande do norte":"RN",
    "rio grande do sul":"RS","rondonia":"RO","rondônia":"RO","roraima":"RR","santa catarina":"SC","sao paulo":"SP","são paulo":"SP",
    "sergipe":"SE","tocantins":"TO"
}

def uf_to_sigla(x):
    if pd.isna(x): return pd.NA
    s = str(x).strip().lower()
    return UF_SIGLA.get(s, s.upper() if len(s)==2 else pd.NA)

ingestion_ts = datetime.datetime.utcnow()

# ---------- 4) Ocorrências ----------
sheet_occ = find_sheet(xls, "Ocorrências")
if not sheet_occ:
    raise ValueError("Aba 'Ocorrências' não encontrada no arquivo UF.")

pdf_occ = pd.read_excel(xls, sheet_name=sheet_occ, dtype=str)
# mapeia nomes de colunas
map_occ = {
    "UF":"uf_nome",
    "Tipo Crime":"tipo_crime",
    "Ano":"year",
    "Mês":"month_text",
    "Ocorrências":"ocorrencias"
}
cols = {c: map_occ.get(c, c) for c in pdf_occ.columns}
pdf_occ.rename(columns=cols, inplace=True)
# normaliza
pdf_occ["uf_nome"]    = pdf_occ["uf_nome"].astype(str).str.strip()
pdf_occ["uf_sigla"]   = pdf_occ["uf_nome"].apply(uf_to_sigla)
pdf_occ["tipo_crime"] = pdf_occ["tipo_crime"].astype(str).str.strip()

pdf_occ["year"]  = pd.to_numeric(pdf_occ["year"], errors="coerce").astype("Int64")
pdf_occ["month"] = pdf_occ["month_text"].apply(month_from_text).astype("Int64")
pdf_occ.drop(columns=["month_text"], inplace=True)

pdf_occ["ocorrencias"] = pd.to_numeric(pdf_occ["ocorrencias"], errors="coerce").astype("Int64")
pdf_occ["year_month"]  = pd.to_datetime(
    pdf_occ["year"].astype("string") + "-" + pdf_occ["month"].astype("string").str.zfill(2) + "-01",
    errors="coerce"
)

pdf_occ["_ingestion_ts"] = ingestion_ts
pdf_occ["_source_path"]  = latest_path

# checks leves (não negativos, UF reconhecida)
before = len(pdf_occ)
pdf_occ = pdf_occ[(pdf_occ["uf_sigla"].notna()) & (pdf_occ["ocorrencias"].isna() | (pdf_occ["ocorrencias"] >= 0))]
after  = len(pdf_occ)
print(f"[Ocorrências] linhas removidas por checks leves: {before - after}")

# envia ao Spark e escreve
df_occ = (spark.createDataFrame(pdf_occ.astype(object))
          .withColumn("year",       F.col("year").cast(T.IntegerType()))
          .withColumn("month",      F.col("month").cast(T.IntegerType()))
          .withColumn("year_month", F.to_date("year_month"))
          .withColumn("ocorrencias",F.col("ocorrencias").cast(T.IntegerType()))
          .withColumn("_ingestion_ts", F.col("_ingestion_ts").cast(T.TimestampType()))
         )

tbl_occ = f"{CATALOG}.{SCHEMA_BRONZE}.ufocorrencias"
(df_occ.write
   .format("delta")
   .mode("overwrite")     # após a 1ª carga, use 'append'
   .option("overwriteSchema","true")
   .partitionBy("uf_sigla","year","month")
   .saveAsTable(tbl_occ))
print(f"[ok] gravado: {tbl_occ} | linhas: {df_occ.count()}")

# ---------- 5) Vítimas ----------
sheet_vit = find_sheet(xls, "Vítimas")
if not sheet_vit:
    raise ValueError("Aba 'Vítimas' não encontrada no arquivo UF.")

pdf_vit = pd.read_excel(xls, sheet_name=sheet_vit, dtype=str)
map_vit = {
    "UF":"uf_nome",
    "Tipo Crime":"tipo_crime",
    "Ano":"year",
    "Mês":"month_text",
    "Sexo da Vítima":"sexo_vitima",
    "Vítimas":"vitimas"
}
cols = {c: map_vit.get(c, c) for c in pdf_vit.columns}
pdf_vit.rename(columns=cols, inplace=True)

pdf_vit["uf_nome"]    = pdf_vit["uf_nome"].astype(str).str.strip()
pdf_vit["uf_sigla"]   = pdf_vit["uf_nome"].apply(uf_to_sigla)
pdf_vit["tipo_crime"] = pdf_vit["tipo_crime"].astype(str).str.strip()
pdf_vit["sexo_vitima"]= pdf_vit["sexo_vitima"].astype(str).str.strip()

pdf_vit["year"]  = pd.to_numeric(pdf_vit["year"], errors="coerce").astype("Int64")
pdf_vit["month"] = pdf_vit["month_text"].apply(month_from_text).astype("Int64")
pdf_vit.drop(columns=["month_text"], inplace=True)

pdf_vit["vitimas"]    = pd.to_numeric(pdf_vit["vitimas"], errors="coerce").astype("Int64")
pdf_vit["year_month"] = pd.to_datetime(
    pdf_vit["year"].astype("string") + "-" + pdf_vit["month"].astype("string").str.zfill(2) + "-01",
    errors="coerce"
)

pdf_vit["_ingestion_ts"] = ingestion_ts
pdf_vit["_source_path"]  = latest_path

# checks leves
before = len(pdf_vit)
pdf_vit = pdf_vit[(pdf_vit["uf_sigla"].notna()) & (pdf_vit["vitimas"].isna() | (pdf_vit["vitimas"] >= 0))]
after  = len(pdf_vit)
print(f"[Vítimas] linhas removidas por checks leves: {before - after}")

df_vit = (spark.createDataFrame(pdf_vit.astype(object))
          .withColumn("year",       F.col("year").cast(T.IntegerType()))
          .withColumn("month",      F.col("month").cast(T.IntegerType()))
          .withColumn("year_month", F.to_date("year_month"))
          .withColumn("vitimas",    F.col("vitimas").cast(T.IntegerType()))
          .withColumn("_ingestion_ts", F.col("_ingestion_ts").cast(T.TimestampType()))
         )

tbl_vit = f"{CATALOG}.{SCHEMA_BRONZE}.ufvitimas"
(df_vit.write
   .format("delta")
   .mode("overwrite")     # após a 1ª carga, use 'append'
   .option("overwriteSchema","true")
   .partitionBy("uf_sigla","year","month")
   .saveAsTable(tbl_vit))
print(f"[ok] gravado: {tbl_vit} | linhas: {df_vit.count()}")